To use the examples in this chapter, first run the code below to import the PyCh Library.

In [1]:
from PyCh import *

PyCh version 0.1 imported succesfully.
 


# 7 Processes

Pych has been designed for modeling and analyzing systems with many components, all working together to obtain the total system behavior. Each component exhibits behavior over time. Sometimes they are busy making internal decisions, sometimes they interact with other components. The language uses a process to model the behavior of a component (the primary interest are the actions of the component rather than its physical representation). This leads to models with many processes working in parallel (also known as concurrent processes), interacting with each other.

Another characteristic of these systems is that the parallelism happens at different scales at the same time, and each scale can be considered to be a collection of co-operating parallel working processes.For example, a factory can be seen as a single component, it accepts supplies and delivers products. However, within a factory, you can have several parallel operating production lines, and a line consists of several parallel operating machines. A machine again consists of parallel operating parts. In the other direction, a factory is a small element in a supply chain. Each supply chain is an element in a (distribution) network. Depending on the area that needs to be analyzed, and the level of detail, some scales are precisely modeled, while others either fall outside the scope of the system or are modeled in an abstract way.

In all these systems, the interaction between processes is not random, they understand each other and exchange information. In other words, they communicate with each other. Pych uses channels to model the communication. A channel connects a sending process to a receiving process, allowing the sender to pass messages to the receiver. This chapter discusses parallel operating processes only, communication between processes using channels is discussed in Chapter 8.

As discussed above, a process can be seen as a single component with behavior over time, or as a wrapper around many processes that work at a smaller scale. Pych supports both kinds of processes. The former is modeled with the statements explained in previous chapters and communication that will be explained in the next chapter. The latter (a process as a wrapper around many smaller-scale processes) is supported with the `run` statement.

## 7.1 A single process
The simplest form of processes is a model with one process, as seen in the example below.

A process definition is denoted by the decorator `@process` above the function definition. The process `P` has as only argument `env`, which is the simulation environment in which the process lives. The environment must always be included as the first argument of a process.

Process `P` contains two statements, the first is `yield`, which denotes that the process generates an event and waits till it has been executed. In this instance, the event `env.timeout(1)` denotes a timeout of one second in simulation time, but more on that later. A process must always yield at least one event. The second statement is the `print` statement to output text on the screen.

Our model has simulation environment `env`, and contains one process `P1`. The environment is simulated using `env.run()`. The model is executed by calling `model()`.

In [2]:
@process
def P(env):
    yield env.timeout(1)
    print("Hello. I am process.")

def model():
    env = Environment()
    P1 = P(env)
    env.run()
    
model()

Hello. I am process.


---
The model environment can contain multiple instances created using the same process definition. When simulating the environment using `env.run()`, the simulation continues until all processes are finished executing. To demonstrate, below is an example of a model with two processes `P1` and `P2`.

In [3]:
@process
def P(env, i):
    yield env.timeout(1)
    print("I am process %d" % i)

def model():
    env = Environment()
    P1 = P(env, 1)
    P2 = P(env, 2)
    env.run()
    
model()

I am process 1
I am process 2


## 7.2 Process in process

A process can create other processes, and it can even wait until another process has finished through the `yield` statement (although it can also continue unhalted). In the example below process `P` creates processes P1 and P2, and waits till they are finished to continue. The concept of 'a process in a process' is very useful in keeping the model structured.

In [4]:
@process
def P_a(env):
    print("Start process 1")
    P1 = P_b(env, 1)
    yield P1
    print("Start process 2")
    P2 = P_b(env, 2)
    yield P2
    
@process
def P_b(env, i):
    yield env.timeout(1)
    print("Finished process %d" % i)
    
def model():
    env = Environment()
    P = P_a(env)
    env.run()
    
model()

Start process 1
Finished process 1
Start process 2
Finished process 2


---
Unlike the timeout event, a process starts running when it is defined, not when we yield it. Suppose we redefine `P_a` as seen below. If we run our model again, we can see that the order of execution is changed. 

In [5]:
@process
def P_a(env):
    print("Start process 1")
    P1 = P_b(env, 1)
    P2 = P_b(env, 2)
    yield P1
    print("Start process 2")
    yield P2
    
model()

Start process 1
Finished process 1
Finished process 2
Start process 2


## 7.3 Many processes
Some models consist of many similar processes. In Python we can utilize list comprehension to quickly create many processes. Below is an example of using list comprehension to create 10 instances of `P` at once.

In [6]:
@process
def P(env, i):
    yield env.timeout(1)
    print("I am process %d" % i)

def model():
    env = Environment()
    Processes = [P(env, i) for i in range(10)]
    env.run()
    
model()

I am process 0
I am process 1
I am process 2
I am process 3
I am process 4
I am process 5
I am process 6
I am process 7
I am process 8
I am process 9


## Appendix 7A: process execution order

Processes can be running parallel at the same time. If multiple processes generate an event simultaneously, the event which was scheduled first goes first (the event is scheduled as soon as the process reaches the `yield` statement). In the below example, the events are scheduled simultaneously, in which case the event of the process which was defined first (process `P1`) is executed first.

In [7]:
@process
def P(env, i):
    yield env.timeout(1)
    print("Process %d is finished at time %.1f" % (i, env.now))

def model():
    env = Environment()
    P1 = P(env, 1)
    P2 = P(env, 2)
    env.run()
    
model()

Process 1 is finished at time 1.0
Process 2 is finished at time 1.0


---
Below we have an example in which the events are not generated simultaneously. In the example, we expect process P2 to be finished first, as its event is scheduled before the second event of process P1.

In [8]:
@process
def P_a(env, i):
    yield env.timeout(0.5)
    print("Process %d is halfway done at time %.1f" % (i, env.now))
    yield env.timeout(0.5)
    print("Process %d is finished at time %.1f" % (i, env.now))

@process
def P_b(env, i):
    yield env.timeout(1)
    print("Process %d is finished at time %.1f" % (i, env.now))

def model():
    env = Environment()
    P1 = P_a(env, 1)
    P2 = P_b(env, 2)
    env.run()
    
model()

Process 1 is halfway done at time 0.5
Process 2 is finished at time 1.0
Process 1 is finished at time 1.0
